In [4]:
import pandas as pd
import netCDF4 as nc
import numpy as np
import cv2
import os
import errno
import geopandas as gpd
import time
from pyproj import Geod
from shapely.geometry import shape, Polygon, Point, MultiPoint, box

In [123]:
# LIBRERIAS CREADAS
def crear_carp(direccion):
    try:
        os.mkdir(direccion)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

def data_lec(direccion,datos):
    names = ['date', 'time', 'lat', 'lon', 'MWS', 'CPSL','ERMWS', 'R34', 'R50', 'R64', 'R100', 'R']
    return pd.read_csv(direccion+'/'+datos, sep=",", skip_blank_lines=True, header = None, names = names)

def formato_fecha(df1):
    df = pd.to_datetime(df1['date'], format='%Y%m%d').apply(lambda x: pd.Series([x.year,x.month,x.day], index = ['yy', 'mm', 'dd']))
    return pd.concat([df,df1], axis = 1)

def formato_hora(df):
    return df.assign(time = (df['time']/100).astype(int))

def cambiar_sufijos(lon1):
    if lon1[-1] == 'W':
        lon1 = lon1.replace("W","")
        lon1 = float(lon1) * -1
    else:
        lon1 = lon1.replace("E","").replace("N","")
        lon1 = float(lon1)
    return lon1

def abrir_imagen(timestamp):
    ir_image = nc.Dataset('D:/IR/merg_'+ timestamp +'_4km-pixel.nc4.nc4')
    return np.array(ir_image.variables['Tb'][0,:,:] - 273.15)

def correccion_valor_minimo(image):
    temp_min = np.min(image[np.where(image != np.min(image))])
    image[np.where(image == np.min(image))] = temp_min - 1
    return image 

def binarizar_imagen(img, umbral: float):
    imagen = img.copy()
    pixel_mayor_umbral = np.where(imagen>umbral)
    pixel_valor_nulo = np.where(imagen == np.min(imagen))
    imagen[pixel_mayor_umbral or pixel_valor_nulo] = 0
    pixel_valido = np.where(imagen != 0)
    imagen[pixel_valido] = 255
    return imagen

def crear_circulo(centro, radio):
    angulo = np.arange(0,np.pi*2,0.01)
    coor_x = radio*np.cos(angulo)+centro[0]
    coor_y = radio*np.sin(angulo)+centro[1]
    return (coor_x,coor_y)

def poligono_circulo(coordenadas): 
    return Polygon(list(zip(coordenadas[0],coordenadas[1])))

def contornos_numpy(array):
    nubM = []
    for c in array: nubM.append(c[0].tolist())
    return np.array(nubM)

def convcoor(nube, dlat= 0.036388397, dlon = 0.036392212):
    lonc1 = np.round(np.transpose(((dlon*nube[:,0])-130)),2)
    latc1 = np.round((dlat*nube[:,1]),2)
    return list(zip(lonc1,latc1))

def gdf_conver(gdf):
    """ Función para convertir una serie de datos al crs EPSG:4326, que
    representa una proyección geografica WGS84"""
    return gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[gdf])

def gdf_explode(gdf):
    """ Separación (explode) de una GeoDataFrame en columnas"""
    return gdf.explode().reset_index(drop=True).rename(columns={0: 'geometry'})

def rectificacion(selcon,ctpos,r,polyar):
    rec_inf = Polygon([(-130,0),(-130,15),(-10,15),(-10,0)]) #itcz
    a = rec_inf.intersects(polyar)
    pol_cast = Polygon([(-130,0),(-130,15),(ctpos[0]-r,15),(ctpos[0]-r,ctpos[1]-r),
                        (ctpos[0]+r,ctpos[1]-r),(ctpos[0]+r,15),(-10,15),(-10,0)])
    pol_cast = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[pol_cast])
    if a == True:
        filtro2 = gpd.overlay(selcon, pol_cast, how='difference')
        return filtro2
    else:
        if ctpos[1] > 15:
            pol_cast = Polygon([(-130,0),(-130,15.1),(ctpos[0]-r,15.1),(ctpos[0]-r,ctpos[1]-r),
                        (ctpos[0]+r,ctpos[1]-r),(ctpos[0]+r,15.1),(-10,15.1),(-10,0)])
            pol_cast = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[pol_cast])
            filtro2 = gpd.overlay(selcon, pol_cast, how='difference')
            return filtro2
        else:
            return selcon
        
def correccion_por_df_vacio(df, data):
    if df.empty:
        df['geometry'] = None
        df['ID']= (np.ones(df.shape[0])*i).astype(int)
        return df
    else:
        result = rectificacion(df,data.center_position[i],data.R[i]/111.1, data.poly_rout[i])
        result['ID']= (np.ones(result.shape[0])*i).astype(int)
        result = gdf_explode(result)
        indx = result['geometry'].apply(lambda nube: nube.intersects(data.poly_rout[i]))
        return result[indx]

def each6h(df):
    df = df.loc[(df.time == 0)|(df.time == 12)|(df.time == 18)|(df.time == 6)].reset_index(drop=True)
    return df


In [6]:
cuenca = input("Nombre de la cuenca: ")
direccion_database = 'D:/'+cuenca+'_complete'

Nombre de la cuenca: NA


In [7]:
crear_carp(direccion_database)
lista = pd.read_csv('D:/listado'+cuenca+'.dat', header =  None)

In [8]:
data = data_lec('D:/S7/'+cuenca,'AL012010.dat')

In [9]:
TC = (data
     .pipe(formato_fecha)
     .pipe(formato_hora)
     .assign(lon = data.lon.apply(lambda lon: cambiar_sufijos(lon)))
     .assign(lat = data.lat.apply(lambda lat: cambiar_sufijos(lat)))
     .drop(columns = ['CPSL','ERMWS','R50','R64','R100'])
     )
TC_concat = TC.copy()

In [10]:
from pyproj import Transformer
transformer = Transformer.from_crs(4326, 3857, always_xy=True)

In [11]:
timestamp = (TC_concat[['yy','mm','dd','time']]
             .astype('str')
             .apply(lambda x: x[0]+ x[1].zfill(2) + x[2].zfill(2) + x[3].zfill(2), axis = 1))
center_position = TC_concat[['lon','lat']].apply(lambda x: [x[0],x[1]], axis = 1)

In [12]:
TC_concat = TC_concat.assign(timestamp = timestamp).assign(center_position = center_position).pipe(each6h)
TC_concat['poly_rout'] = (TC_concat[['center_position','R']]
                          .apply(lambda x: crear_circulo(x[0], x[1]/111.1),axis = 1)
                          .apply(poligono_circulo)
                         )


In [13]:
x = []
for pt in transformer.itransform(TC_concat.center_position.to_numpy()):
    x.append(pt)
TC_concat = TC_concat.assign(ctpos_utm = x)

In [14]:
def contornos_df(array):
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    return pd.DataFrame(array, columns = ['array'])

In [15]:
data_base = gpd.GeoDataFrame(columns = ['geometry', 'ID'], crs = 'epsg:4326')
for i in range(TC_concat.shape[0]): 
    IR_img = abrir_imagen(TC_concat.timestamp[i])
    pic1 = IR_img.copy()
    IR_img_corregida = correccion_valor_minimo(pic1)
    IR_img_binarizada = binarizar_imagen(IR_img_corregida, -40)
    (contornos,_) = cv2.findContours(np.uint8(IR_img_binarizada), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    df_contornos = contornos_df(contornos)
    df_contornos['len'] = df_contornos.array.apply(lambda x: len(x))
    df_contornos_sel = df_contornos[df_contornos.len > 10]
    selcon = df_contornos_sel.array.apply(lambda x: contornos_numpy(x)).to_frame(name = 'geometry')
    selcon['geometry'] = selcon['geometry'].apply(convcoor).apply(Polygon)
    selcon = gpd.GeoDataFrame(selcon,geometry='geometry',crs='EPSG:4326')
    contornos_dentro_campo_vientos = (selcon[selcon['geometry']
                                             .apply(lambda nube: nube.intersects(TC_concat.poly_rout[i]))])
    contornos_corregidos = correccion_por_df_vacio(contornos_dentro_campo_vientos, TC_concat)
    
    data_base = pd.concat([data_base,contornos_corregidos], axis=0).reset_index(drop=True)
    
    

In [16]:
def seleccionar_poligonos_800_km_2(data):
    data_base['area'] = data_base['geometry'].apply(areapoly)
    polygons = data_base.loc[data_base['area']>= 800].reset_index(drop= True)
    return polygons

def areapoly(geom):
    """Calculo del area de un poligono con valores en Km^2"""
    geod = Geod(ellps="WGS84")
    area = abs(geod.geometry_area_perimeter(geom)[0])/1e+6
    return area

In [204]:
def multipoint(x, points2):
    """Conversión de un shape en formato multipoint"""
    y = points2.index[(points2['geometry']) == x][0]
    prueba = type(shape(points2['geometry'].iloc[y])) is Polygon
    if prueba is True:
        pnts = MultiPoint(list(x.exterior.coords))
    else:
        pnts = MultiPoint(list(shape(x).coords))
    return pnts

In [18]:
db_poligonos = data_base.pipe(seleccionar_poligonos_800_km_2)

In [19]:
def puntos(ID):
    prueba = gb[gb.ID.eq(0)].reset_index(drop=True)
    prueba['geometry'] = prueba.geometry.apply(multipoint, args=(prueba,))
    return prueba.pipe(gdf_explode)

In [20]:
def intento(df):
    df.assign(x = df.geometry.apply(lambda punto: punto.x)).assign(y = df.geometry.apply(lambda punto: punto.y))
    y = []
    for pt in transformer.itransform(dff.geometry.apply(lambda punto: [punto.x,punto.y])):
        y.append(pt)
    return dff.assign(ctpos_utm = y)

In [21]:
def promedio_si(x):
    return x[x > 0].mean()

In [22]:
radios = TC_concat[['lon','lat','center_position','ctpos_utm']].reset_index(drop = False)
gb = db_poligonos.groupby('ID').apply(lambda x: gdf_explode(x))
for index in radios.index:
    prueba = gb.copy().loc[index,:]
    prueba['geometry'] = prueba.geometry.apply(multipoint, args=(prueba,))

    df = prueba.pipe(gdf_explode)
    if TC_concat.lat[index] <= 35:
        try:
            dff = (df
                   .assign(x = df.geometry.apply(lambda punto: punto.x))
                   .assign(y = df.geometry.apply(lambda punto: punto.y)))
            y = []
            for pt in transformer.itransform(dff.geometry.apply(lambda punto: [punto.x,punto.y])):
                y.append(pt)
            dff = dff.assign(ctpos_utm = y)
            
            dff_ne = dff[dff.x.ge(TC_concat.lon[index]) & dff.y.ge(TC_concat.lat[index])]
            dff_no = dff[dff.x.le(TC_concat.lon[index]) & dff.y.ge(TC_concat.lat[index])]
            dff_so = dff[dff.x.le(TC_concat.lon[index]) & dff.y.le(TC_concat.lat[index])]
            dff_se = dff[dff.x.ge(TC_concat.lon[index]) & dff.y.le(TC_concat.lat[index])]

            distancia_km = lambda x: Point(x).distance(Point(TC_concat.ctpos_utm[index]))
            radios.loc[index, 'rne_km'] = dff_ne['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()
            radios.loc[index, 'rno_km'] = dff_no['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()
            radios.loc[index, 'rso_km'] = dff_so['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()
            radios.loc[index, 'rse_km'] = dff_se['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()

            distancia_grados = lambda x: x.distance(Point(TC_concat.center_position[index]))
            radios.loc[index, 'rne_g'] = dff_ne['geometry'].apply(distancia_grados).max().round(1)
            radios.loc[index, 'rno_g'] = dff_no['geometry'].apply(distancia_grados).max().round(1)
            radios.loc[index, 'rso_g'] = dff_so['geometry'].apply(distancia_grados).max().round(1)
            radios.loc[index, 'rse_g'] = dff_se['geometry'].apply(distancia_grados).max().round(1)
        except: 
            radios.loc[index, 'rne_km'] = -99999
            radios.loc[index, 'rno_km'] = -99999
            radios.loc[index, 'rso_km'] = -99999
            radios.loc[index, 'rse_km'] = -99999
            radios.loc[index, 'rne_g'] = -99999
            radios.loc[index, 'rno_g'] = -99999
            radios.loc[index, 'rso_g'] = -99999
            radios.loc[index, 'rse_g'] = -99999

    else:
        radios.loc[index, 'rne_km'] = -99999
        radios.loc[index, 'rno_km'] = -99999
        radios.loc[index, 'rso_km'] = -99999
        radios.loc[index, 'rse_km'] = -99999
        radios.loc[index, 'rne_g'] = -99999
        radios.loc[index, 'rno_g'] = -99999
        radios.loc[index, 'rso_g'] = -99999
        radios.loc[index, 'rse_g'] = -99999
        
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [23]:
radios['rp_km'] = radios[['rne_km','rno_km','rso_km','rse_km']].apply(lambda x: promedio_si(x), axis =1).round(2)
radios['rp_g'] = radios[['rne_g','rno_g','rso_g','rse_g']].apply(lambda x: promedio_si(x), axis =1).round(1)

In [24]:
## 6.1 Asimetría
def assym(data):
    return np.round(data[['rne_km','rno_km','rso_km','rse_km']].apply(
                     lambda x: (x.max()-x.min())/x.max(),axis=1),2)
## 6.2 Dispersion
def disp(ct,poly):
    poly_fit = poly.reset_index(drop=True)
    Ai = poly_fit.groupby(['ID']).sum().loc[:, 'area']
    poly_fit['centroid'] = poly_fit['geometry'].to_crs('epsg:3785').centroid.to_crs(poly_fit['geometry'].crs)
    poly_fit['ctpos'] = poly_fit['ID'].apply(lambda i: Point(ct['lon'][i], ct['lat'][i]))
    poly_fit['Aid'] = poly_fit['ID'].apply(lambda x: Ai[x])
    r_cent = poly_fit[['centroid','ctpos']].apply(lambda x: distancia(x[0] ,x[1]),axis =1)
    r_cuad = poly_fit[['centroid','ctpos','ID']].apply(lambda x: cuad(ct,x[0],x[1],x[2]), axis=1)
    poly_fit['rc/r'] = r_cent/r_cuad
    poly_fit['Ai/Aid'] = poly_fit[['area','Aid']].apply(lambda x: x[0]/x[1], axis =1)
    poly_fit['A'] = poly_fit['rc/r'] *poly_fit['Ai/Aid']
    return np.round(poly_fit.groupby(['ID']).sum().loc[:, 'A'],2)

def distancia(psel,center):
    d = center.distance(psel)*111.11
    return d
def cuad(df,cent, ctpp,i):
    cuadrantes = ['rne_km','rno_km','rso_km','rse_km']
    boxes = [box(ctpp.x,ctpp.y,-10,40),box(ctpp.x,ctpp.y,-130,40),
             box(ctpp.x,ctpp.y,-130,0), box(ctpp.x,ctpp.y,-10,0)]
    for ind,b in enumerate(boxes):
        if b.contains(cent) == True:
            radios = cuadrantes[ind]
            return df[cuadrantes[ind]].loc[i]
## 6.3 Solidez
def soli(df,pol):
    Ax = df[['lon','lat','rne_km','rno_km','rso_km','rse_km']].apply(
        lambda x: polyarc(x[0],x[1],x[2],x[3],x[4],x[5]), axis=1)
    Ai = pol.groupby(['ID']).sum().loc[:, 'area']
    return np.round(Ai/Ax,2)

def ctfield(r1,r2,r3,r4,c):
    ang = [np.arange(0,np.pi/2,0.01), np.arange(np.pi/2,np.pi,0.01),
           np.arange(np.pi,(3/2)*np.pi,0.01),np.arange((3/2)*np.pi,2*np.pi,0.01)]
    grafx, grafy = [], []
    for i,r in enumerate([r1,r2,r3,r4]):
        grafx.append(r*np.cos(ang[i])+c[0])
        grafy.append(r*np.sin(ang[i])+c[1])
    return (np.concatenate(grafx, axis = 0),np.concatenate(grafy, axis = 0))

def polyarc(lon,lat,r1,r2,r3,r4):
    ctpos = [lon,lat]
    u=111.1
    arc = ctfield(r1/u,r2/u,r3/u,r4/u,ctpos)
    polyar = Polygon(list(zip(arc[0],arc[1])))
    area = areapoly(polyar)
    return area


In [25]:
radios['A'] = assym(radios)
radios['D'] = disp(radios,gb)
radios['S'] = soli(radios,gb.reset_index(drop=True))

In [26]:
tabla_semifinal = TC_concat.iloc[:,[3,4,5,6,7]].join(radios.iloc[:, 5:])

In [78]:
def abrir_imerg(filename):
    ds = nc.Dataset(filename)
    pcp = ds.variables['precipitationCal'][0,:,:]
    imagen = np.array(pcp)
    pic1= imagen[:,:]
    return np.transpose(pic1)

In [100]:
def bina(imagen,umbral):
    """ Función para binarizar la imagen en función de un umbral determinado"""
    minimo = np.min(imagen)
    indx = np.where(imagen<umbral)
    imagen[indx] = 0
    ind = np.where(imagen == minimo)
    imagen[ind] = 0
    indi = np.where(imagen != 0)
    imagen[indi] = 255
    return imagen

In [88]:
def convcoor_rain(nube,dlat= 0.099999994, dlon = 0.1000061):
    """ Función para determinar las coordenadas geograficas de los contornos
    calculados en el programa [contornos]. Los parametros de entrada son:
    nube = la cual representa cada segmento de contornos
    dlat, dlon = son medidas para redefinir los saltos de malla"""
    lonc1 = np.round(np.transpose(((dlon*nube[:,0])-130)),2)
    latc1 = np.round((dlat*nube[:,1]),2)
    coordenadas = list(zip(lonc1,latc1))
    return coordenadas

In [188]:
def arco(r1,r2,r3,r4,c):
    radi = []
    for radio in [r1,r2,r3,r4]:
        if radio < 0:
            radio = 0
            radi.append(radio)
        else: 
            radio = radio
            radi.append(radio)
            
    ang = [np.arange(0,np.pi/2,0.01), np.arange(np.pi/2,np.pi,0.01),
           np.arange(np.pi,(3/2)*np.pi,0.01),np.arange((3/2)*np.pi,2*np.pi,0.01)]
    grafx, grafy = [], []
    for i,r in enumerate(radi):
        grafx.append(r*np.cos(ang[i])+c[0])
        grafy.append(r*np.sin(ang[i])+c[1])
    return (np.concatenate(grafx, axis = 0),np.concatenate(grafy, axis = 0))

In [244]:
def correccion_por_df_vacio_rain(df, data):
    if df.empty:
        df['geometry'] = None
        df['ID']= (np.ones(df.shape[0])*i).astype(int)
        return df
    else:
        result = rectificacion(df,data.center_position[index],data.rp_g[index], data.poly_cuad[index])
        result['ID']= (np.ones(result.shape[0])*index).astype(int)
        result = gdf_explode(result)
        indx = result['geometry'].apply(lambda nube: nube.intersects(data.poly_cuad[index]))
        return result[indx]

In [217]:
index = 0


In [248]:
data_rain = TC_concat.copy()
data_rain['name_id'] = data_rain[['date','time']].apply(lambda x: str(x[0])+'-S'+str(x[1]).zfill(2), axis = 1)
data_rain['filename'] = data_rain[['name_id','time']].apply(lambda x: 'D:/GPM_IMERG/3B-HHR.MS.MRG.3IMERG.'+x[0]+'0000-E'+str(x[1]).zfill(2)+'2959.'+str(60*x[1]).zfill(4)+'.V06B.HDF5.nc4', axis = 1)
data_rain['poly_cuad'] = radios[['center_position','rne_g','rno_g', 'rso_g', 'rse_g']].apply(lambda x: arco(x[1],x[2],x[3],x[4],x[0]), axis =1).apply(poligono_circulo)
for index in range(data_rain.shape[0]):
    print(index)
    imerg_img = abrir_imerg(data_rain.filename[index])
    pic1 = imerg_img.copy()
    rain_img_corregida = correccion_valor_minimo(pic1)
    rain_img_binarizada = bina(rain_img_corregida, 2.5)
    (contornos,_) = cv2.findContours(np.uint8(rain_img_binarizada), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    df_contornos = contornos_df(contornos)
    df_contornos['len'] = df_contornos.array.apply(lambda x: len(x))
    df_contornos_sel = df_contornos[df_contornos.len > 10]
    selcon = df_contornos_sel.array.apply(lambda x: contornos_numpy(x)).to_frame(name = 'geometry')
    selcon['geometry'] = selcon['geometry'].apply(convcoor_rain).apply(Polygon)
    selcon = gpd.GeoDataFrame(selcon,geometry='geometry',crs='EPSG:4326')
    contornos_dentro_campo_vientos = (selcon[selcon['geometry']
                                         .apply(lambda nube: nube.intersects(data_rain.poly_cuad[index]))])
    contornos_corregidos = correccion_por_df_vacio_rain(contornos_dentro_campo_vientos, radios)
    polygons_sel = contornos_corregidos.pipe(gdf_explode)
    polygons_sel['geometry'] = polygons_sel.geometry.apply(multipoint, args=(polygons_sel,))
    df = polygons_sel.pipe(gdf_explode)
    if TC_concat.lat[index] <= 35:
        try:
            dff = (df
                   .assign(x = df.geometry.apply(lambda punto: punto.x))
                   .assign(y = df.geometry.apply(lambda punto: punto.y)))
            y = []
            for pt in transformer.itransform(dff.geometry.apply(lambda punto: [punto.x,punto.y])):
                y.append(pt)
            dff = dff.assign(ctpos_utm = y)
            
            dff_ne = dff[dff.x.ge(TC_concat.lon[index]) & dff.y.ge(TC_concat.lat[index])]
            dff_no = dff[dff.x.le(TC_concat.lon[index]) & dff.y.ge(TC_concat.lat[index])]
            dff_so = dff[dff.x.le(TC_concat.lon[index]) & dff.y.le(TC_concat.lat[index])]
            dff_se = dff[dff.x.ge(TC_concat.lon[index]) & dff.y.le(TC_concat.lat[index])]

            distancia_km = lambda x: Point(x).distance(Point(TC_concat.ctpos_utm[index]))
            radios.loc[index, 'rne_rpb_km'] = dff_ne['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()
            radios.loc[index, 'rno_rpb_km'] = dff_no['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()
            radios.loc[index, 'rso_rpb_km'] = dff_so['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()
            radios.loc[index, 'rse_rpb_km'] = dff_se['ctpos_utm'].apply(distancia_km).div(1000).round(2).max()
        except: 
            radios.loc[index, 'rne_rpb_km'] = -99999
            radios.loc[index, 'rno_rpb_km'] = -99999
            radios.loc[index, 'rso_rpb_km'] = -99999
            radios.loc[index, 'rse_rpb_km'] = -99999
    else:
        radios.loc[index, 'rne_rpb_km'] = -99999
        radios.loc[index, 'rno_rpb_km'] = -99999
        radios.loc[index, 'rso_rpb_km'] = -99999
        radios.loc[index, 'rse_rpb_km'] = -99999


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [245]:
index = 29
imerg_img = abrir_imerg(data_rain.filename[index])
pic1 = imerg_img.copy()
rain_img_corregida = correccion_valor_minimo(pic1)
rain_img_binarizada = bina(rain_img_corregida, 2.5)
(contornos,_) = cv2.findContours(np.uint8(rain_img_binarizada), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
df_contornos = contornos_df(contornos)
df_contornos['len'] = df_contornos.array.apply(lambda x: len(x))
df_contornos_sel = df_contornos[df_contornos.len > 10]
selcon = df_contornos_sel.array.apply(lambda x: contornos_numpy(x)).to_frame(name = 'geometry')
selcon['geometry'] = selcon['geometry'].apply(convcoor_rain).apply(Polygon)
selcon = gpd.GeoDataFrame(selcon,geometry='geometry',crs='EPSG:4326')
contornos_dentro_campo_vientos = (selcon[selcon['geometry']
                                     .apply(lambda nube: nube.intersects(data_rain.poly_cuad[index]))])
contornos_corregidos = correccion_por_df_vacio_rain(contornos_dentro_campo_vientos, radios)
polygons_sel = contornos_corregidos.pipe(gdf_explode)
polygons_sel['geometry'] = polygons_sel.geometry.apply(multipoint, args=(polygons_sel,))
df = polygons_sel.pipe(gdf_explode)

In [249]:
radios

,index,lon,lat,center_position,ctpos_utm,rne_km,rno_km,rso_km,rse_km,rne_g,...,rp_km,rp_g,A,D,S,poly_cuad,rne_rpb_km,rno_rpb_km,rso_rpb_km,rse_rpb_km
0,0,-82.0,15.9,"[-82.0, 15.9]","(-9128198.245048434, 1793145.0969010212)",984.00,806.96,524.40,725.81,8.8,...,760.29,6.8,0.47,0.60,0.10,"POLYGON ((-73.2 15.9, -73.20043999633334 15.98...",1220.34,856.85,NaN,1257.96
1,1,-82.1,16.0,"[-82.1, 16.0]","(-9139330.194127759, 1804722.766257293)",1319.75,1107.29,590.17,1313.10,11.8,...,1082.58,9.7,0.55,0.58,0.16,"POLYGON ((-70.3 16, -70.30058999508334 16.1179...",1186.01,1091.72,356.22,1341.38
2,2,-82.3,16.1,"[-82.3, 16.1]","(-9161594.092286414, 1816306.2312880505)",891.03,734.65,764.33,685.37,7.8,...,768.84,6.8,0.23,0.45,0.19,"POLYGON ((-74.5 16.1, -74.50038999675 16.17799...",230.07,679.17,792.00,684.58
3,3,-82.5,16.2,"[-82.5, 16.2]","(-9183857.990445068, 1827895.533119448)",773.81,768.50,871.54,676.42,6.9,...,772.57,6.9,0.22,0.54,0.25,"POLYGON ((-75.59999999999999 16.2, -75.6003449...",724.81,696.57,680.64,384.87
4,4,-83.1,16.4,"[-83.1, 16.4]","(-9250649.684921034, 1851091.8121785242)",1175.57,794.17,696.03,1187.78,10.6,...,963.39,8.6,0.41,0.43,0.22,"POLYGON ((-72.5 16.4, -72.50052999558335 16.50...",1057.79,637.99,987.18,1059.82
5,5,-83.9,16.6,"[-83.9, 16.6]","(-9339705.277555654, 1874311.9344140864)",1173.64,917.27,791.36,1155.50,10.5,...,1009.44,9.0,0.33,0.34,0.30,"POLYGON ((-73.40000000000001 16.6, -73.4005249...",1055.92,888.28,834.76,1039.29
6,6,-84.9,16.7,"[-84.9, 16.7]","(-9451024.768348927, 1885931.0405919396)",785.30,870.65,878.26,768.95,6.8,...,825.79,7.2,0.12,0.33,0.37,"POLYGON ((-78.10000000000001 16.7, -78.1003399...",552.35,400.75,864.21,653.82
7,7,-86.1,16.9,"[-86.1, 16.9]","(-9584608.157300854, 1909187.5516940707)",779.35,840.53,1069.09,852.33,6.7,...,885.32,7.7,0.27,0.22,0.33,"POLYGON ((-79.39999999999999 16.9, -79.4003349...",721.26,841.47,1228.91,848.18
8,8,-87.2,17.2,"[-87.2, 17.2]","(-9707059.597173456, 1944118.6941882686)",883.86,818.92,1009.81,959.95,7.9,...,918.14,8.1,0.19,0.42,0.37,"POLYGON ((-79.3 17.2, -79.30039499670835 17.27...",907.80,449.00,825.23,890.17
9,9,-88.2,17.5,"[-88.2, 17.5]","(-9818379.087966729, 1979106.4997243874)",1035.49,661.40,1050.54,949.56,9.3,...,924.25,8.2,0.37,0.30,0.40,"POLYGON ((-78.90000000000001 17.5, -78.9004649...",1079.16,421.01,1049.86,927.85


In [250]:
radios['rpb_km'] = radios[['rne_rpb_km','rno_rpb_km','rso_rpb_km','rse_rpb_km']].apply(lambda x: promedio_si(x), axis =1).round(2)

In [304]:
tabla_final = tabla_semifinal.join(radios.iloc[:,19:])

In [305]:
tabla_final['MWS'] = (tabla_final['MWS']*0.539957).round(0).astype(int)


In [306]:
tabla_final['time'] = tabla_final['time'].apply(lambda x: str(x).zfill(2)+'00')

In [308]:
tabla_final = tabla_final[['date', 'time', 'lat', 'lon', 'MWS', 'rne_km', 'rno_km', 'rso_km',
                           'rse_km', 'rp_km', 'rne_g', 'rno_g', 'rso_g', 'rse_g', 'rp_g', 'A', 'D',
                           'S', 'rne_rpb_km', 'rno_rpb_km', 'rso_rpb_km', 'rse_rpb_km', 'rpb_km']]

In [299]:
# tabla_final['lat'] = tabla_final['lat'].apply(lambda x: x.format('%2.1f'))
# tabla_final['lon'] = tabla_final['lon'].map(lambda x: '%4.1f'.format)
# tabla_final['rne_km'] = tabla_final['rne_km'].map(lambda x: '%4.2f'.format)
# tabla_final['rno_km'] = tabla_final['rno_km'].map(lambda x: '%4.2f'.format)
# tabla_final['rso_km'] = tabla_final['rso_km'].map(lambda x: '%4.2f'.format)
# tabla_final['rse_km'] = tabla_final['rse_km'].map(lambda x: '%4.2f'.format)
# tabla_final['rp_km'] = tabla_final['rp_km'].map(lambda x: '%4.2f'.format)

# tabla_final['rne_g'] = tabla_final['rne_g'].map(lambda x: '%2.1f'.format)
# tabla_final['rno_g'] = tabla_final['rno_g'].map(lambda x: '%2.1f'.format)
# tabla_final['rso_g'] = tabla_final['rso_g'].map(lambda x: '%2.1f'.format)
# tabla_final['rse_g'] = tabla_final['rse_g'].map(lambda x: '%2.1f'.format)
# tabla_final['rp_g'] = tabla_final['rp_g'].map(lambda x: '%2.1f'.format)

# tabla_final['A'] = tabla_final['A'].map(lambda x: '%1.2f'.format)
# tabla_final['D'] = tabla_final['D'].map(lambda x: '%1.2f'.format)
# tabla_final['S'] = tabla_final['S'].map(lambda x: '%1.2f'.format)

# tabla_final['rne_km'] = tabla_final['rne_km'].map(lambda x: '%4.2f'.format(x))
# tabla_final['rno_km'] = tabla_final['rno_km'].map(lambda x: '%4.2f'.format)
# tabla_final['rso_km'] = tabla_final['rso_km'].map(lambda x: '%4.2f'.format)
# tabla_final['rse_km'] = tabla_final['rse_km'].map(lambda x: '%4.2f'.format)
# tabla_final['rp_km'] = tabla_final['rp_km'].map(lambda x: '%4.2f'.format)

In [309]:
tabla_final

,date,time,lat,lon,MWS,rne_km,rno_km,rso_km,rse_km,rp_km,...,rse_g,rp_g,A,D,S,rne_rpb_km,rno_rpb_km,rso_rpb_km,rse_rpb_km,rpb_km
0,20100624,1800,15.9,-82.0,25,984.00,806.96,524.40,725.81,760.29,...,6.5,6.8,0.47,0.60,0.10,1220.34,856.85,NaN,1257.96,1111.72
1,20100625,0000,16.0,-82.1,25,1319.75,1107.29,590.17,1313.10,1082.58,...,11.8,9.7,0.55,0.58,0.16,1186.01,1091.72,356.22,1341.38,993.83
2,20100625,0600,16.1,-82.3,25,891.03,734.65,764.33,685.37,768.84,...,6.1,6.8,0.23,0.45,0.19,230.07,679.17,792.00,684.58,596.46
3,20100625,1200,16.2,-82.5,30,773.81,768.50,871.54,676.42,772.57,...,6.1,6.9,0.22,0.54,0.25,724.81,696.57,680.64,384.87,621.72
4,20100625,1800,16.4,-83.1,30,1175.57,794.17,696.03,1187.78,963.39,...,10.7,8.6,0.41,0.43,0.22,1057.79,637.99,987.18,1059.82,935.70
5,20100626,0000,16.6,-83.9,30,1173.64,917.27,791.36,1155.50,1009.44,...,10.4,9.0,0.33,0.34,0.30,1055.92,888.28,834.76,1039.29,954.56
6,20100626,0600,16.7,-84.9,40,785.30,870.65,878.26,768.95,825.79,...,6.7,7.2,0.12,0.33,0.37,552.35,400.75,864.21,653.82,617.78
7,20100626,1200,16.9,-86.1,40,779.35,840.53,1069.09,852.33,885.32,...,7.5,7.7,0.27,0.22,0.33,721.26,841.47,1228.91,848.18,909.96
8,20100626,1800,17.2,-87.2,55,883.86,818.92,1009.81,959.95,918.14,...,8.4,8.1,0.19,0.42,0.37,907.80,449.00,825.23,890.17,768.05
9,20100627,0000,17.5,-88.2,55,1035.49,661.40,1050.54,949.56,924.25,...,8.5,8.2,0.37,0.30,0.40,1079.16,421.01,1049.86,927.85,869.47


In [318]:
#tabla_final['lat'] = tabla_final['lat'].map(lambda x: '%2.1f' % x)
tabla_final.to_csv('alex_ws.dat',sep = '\t', na_rep = 0.00, index = False, header = False)